In [1]:
# !pip install -q six numpy scipy matplotlib scikit-image opencv-python imageio
# !pip install -q keras imgaug
# !pip install -q keras

In [2]:
import keras
import imgaug as ia
from imgaug import augmenters as iaa

Using TensorFlow backend.


In [3]:
import numpy as np
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import SGD
from keras.callbacks import *
from keras.models import load_model


# from keras.callbacks import LearningRateScheduler,ModelCheckpoint,EarlyStopping,LambdaCallback
import os,sys,math

In [4]:
import google
colab_dir='./'
file_name='EIP_CIFAR_10'
if hasattr(google,'colab'):
    from google.colab import drive
    drive.mount('/content/gdrive')
    colab_dir='/content/gdrive/My Drive/Colab Notebooks/'
model_file=colab_dir+file_name+'.h5'

In [5]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [6]:
#Augmentation and resizing
#augment and then concat samples with original
def augment(dataset,flip=0.5,blur=1.0):
    ia.seed(1)
#     seq = iaa.Sequential([iaa.Fliplr(flip),iaa.GaussianBlur(sigma=(0, blur)),iaa.Sometimes(iaa.Crop(percent=(0, 0.1))),
#                          iaa.Sometimes(iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
#                                              translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
#                                              rotate=(-45, 45),shear=(-16, 16),order=[0, 1],cval=(0, 255),mode=ia.ALL))])
    seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Crop(percent=(0, 0.1)),
    iaa.Sometimes(0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    iaa.ContrastNormalization((0.75, 1.5)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8))
    ], random_order=True)    
    return seq.augment_images(dataset)

def augmenter(X,y,start=0,end=1,append=True):
    ln=len(X)
    print('Before augmentation:',X.shape,y.shape)
    if not append:
        start=0
        end=1
    start=int(start*ln)
    end=int(end*ln)
    new_X=augment(X)[start:end]
    new_y=y[start:end]
    print(append)
    if append:
        X=np.concatenate((X,new_X))
        y=np.concatenate((y,new_y))
    print('After augmentation:',X.shape,y.shape)
    return (X,y)

#26x26 is almost half of 32x32. 22x22 maybe too small even though its exact half.
def resize_imgs(imgs,shape=(26,26)):
    seq = iaa.Sequential([iaa.Scale({"height": shape[0], "width": shape[1]})])
    return seq.augment_images(imgs)

In [8]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2,num_layers=12):
    global compression
    temp = input
    for _ in range(num_layers):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*2), (1,1),kernel_initializer='he_normal', use_bias=False ,padding='same')(relu)
        BatchNorm = BatchNormalization()(Conv2D_3_3)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter), (3,3),kernel_initializer='he_normal', use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])        
        temp = concat        
    return temp

def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter), (1,1),kernel_initializer='he_normal', use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
        Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)    
    return avg

def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)    
    return output
  
def dense_unit(input_layer,num_filter, dropout_rate,num_layers):
    dense_block=add_denseblock(input_layer, num_filter, dropout_rate,num_layers)
    transition_block=add_transition(dense_block, num_filter, dropout_rate)
    return transition_block
  
def dense_units_chain(n,input_layer,num_filter, dropout_rate,num_layers):
    dense_unit_=input_layer
    for i in range(n):
        dense_unit_=dense_unit(dense_unit_,num_filter, dropout_rate,num_layers)
    return dense_unit_

In [9]:
#callback classes
#try building model with clr, snapshot ensemble

class CyclicLR(Callback):
    "source: https://github.com/bckenstler/CLR/blob/master/clr_callback.py"
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [10]:
#new model with larger i/p layer
def larger_model(src_model,shape=(32,32,3)):
    new_input=Input(shape)
    src_model.layers.pop(0)
    new_output=src_model(new_input)
    new_model=Model(new_input,new_output)
    return new_model

# Load CIFAR10 Data
def load_data(resize=False,shape=(26,26),test_augment=False,append=True):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    if resize:
        x_train=resize_imgs(x_train,shape)
        x_test=resize_imgs(x_test,shape)
    
    (x_train, y_train) = augmenter(x_train, y_train,end=1,append=append)
    if test_augment:
        (x_test, y_test) = augmenter(x_test, y_test,end=1,append=append)
    return (x_train, y_train,x_test, y_test)
#create a dnn model
def create_model(input_shape,num_layers,input=None):
    print('Creating model with input shape',input_shape)
    if input is None:
        input = Input(input_shape)
    First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)
    hidden_dense_blocks = dense_units_chain(n_dense_blocks,First_Conv2D,num_filter,dropout_rate,num_layers)
    Last_Block = add_denseblock(hidden_dense_blocks, num_filter, dropout_rate)
    output = output_layer(Last_Block)
    model = Model(inputs=[input], outputs=[output])
    return model

In [11]:
print('====================HYPER PARAMETERS====================')

====================HYPER PARAMETERS====================


In [12]:
# Hyperparameters
batch_size = 100
num_classes = 10
max_epochs = 250
model_small_epochs=20
model_large_epochs1=100
model_large_epochs2=80

num_layers = 34
# layers_small=12
# layers_large=num_layers-layers_small
layers_large=layers_small=num_layers
num_filter = 12
compression = 0.5
dropout_rate = 0.1#0.2
n_dense_blocks = 3
smaller_input=(26,26,3)

In [13]:
print('====================BEGIN OF LARGER MODEL====================')

====================BEGIN OF LARGER MODEL====================


In [14]:
#load data
x_train, y_train, x_test, y_test = load_data(resize=False,append=False)
#callbacks
model_checkpointer=ModelCheckpoint('large_weights.{epoch:02d}-{val_acc:.2f}.h5', monitor='val_acc',
                verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=2)

clr = CyclicLR(base_lr=0.01, max_lr=0.2,step_size=8*(len(y_train)/batch_size),mode='exp_range',gamma=0.99994)
callbacks = [clr, model_checkpointer]

Before augmentation: (50000, 32, 32, 3) (50000, 10)
False
After augmentation: (50000, 32, 32, 3) (50000, 10)


In [15]:
#create model
model_large = create_model(x_train.shape[1:],num_layers=layers_large)
model_large.compile(loss='categorical_crossentropy',metrics=['accuracy'],
                    optimizer=SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True))
model_large.summary()

Creating model with input shape (32, 32, 3)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 12)   48          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 12)   0           batch_normalization_1[0][0]      
_________________________________________________________________

In [ ]:
#callbacks
clr = CyclicLR(base_lr=0.1, max_lr=0.2,step_size=8*(len(y_train)/batch_size))
callbacks = [clr, model_checkpointer]

#train model
hs=model_large.fit(x_train, y_train, batch_size=batch_size,verbose=1,
                   epochs=model_small_epochs, callbacks=callbacks,
                   validation_data=(x_test, y_test))
model_large.save(model_file)
print('Saved model_small to disk')

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 293s 6ms/step - loss: 2.0976 - acc: 0.2721 - val_loss: 2.0872 - val_acc: 0.2778
Epoch 2/20
50000/50000 [==============================] - 242s 5ms/step - loss: 1.5336 - acc: 0.4394 - val_loss: 1.3654 - val_acc: 0.5053

Epoch 00002: val_acc improved from -inf to 0.50530, saving model to large_weights.02-0.51.h5
Epoch 3/20
50000/50000 [==============================] - 242s 5ms/step - loss: 1.2855 - acc: 0.5358 - val_loss: 1.3545 - val_acc: 0.5193
Epoch 4/20
50000/50000 [==============================] - 242s 5ms/step - loss: 1.1459 - acc: 0.5889 - val_loss: 1.1426 - val_acc: 0.6028

Epoch 00004: val_acc improved from 0.50530 to 0.60280, saving model to large_weights.04-0.60.h5
Epoch 5/20
50000/50000 [==============================] - 243s 5ms/step - loss: 1.0365 - acc: 0.6298 - val_loss: 1.0758 - val_acc: 0.6254
Epoch 6/20
50000/50000 [==============================] - 243s 5ms/st

In [ ]:
# clr = CyclicLR(base_lr=0.01, max_lr=0.2,step_size=8*(len(y_train)/batch_size))
# callbacks = [clr, model_checkpointer]
# #train model
# hs=model_large.fit(x_train, y_train, batch_size=batch_size,verbose=1,
#                     initial_epoch=model_small_epochs,epochs=20+4, callbacks=callbacks,
#                     validation_data=(x_test, y_test))
# model_large.save(model_file)
# print('Saved model_small to disk')

In [ ]:
clr = CyclicLR(base_lr=0.001, max_lr=0.1,step_size=8*(len(y_train)/batch_size))
callbacks = [clr, model_checkpointer]

#train model
hs=model_large.fit(x_train, y_train, batch_size=batch_size,verbose=1,
                    initial_epoch=model_small_epochs+24,epochs=20+24+32, callbacks=callbacks,
                    validation_data=(x_test, y_test))
model_large.save(model_file)
print('Saved model_small to disk')

In [ ]:
clr = CyclicLR(base_lr=0.001, max_lr=0.01,step_size=8*(len(y_train)/batch_size))
callbacks = [clr, model_checkpointer]

#train model
hs=model_large.fit(x_train, y_train, batch_size=batch_size,verbose=1,
                    initial_epoch=76,epochs=76+16, callbacks=callbacks,
                    validation_data=(x_test, y_test))
model_large.save(model_file)
print('Saved model_small to disk')

In [ ]:
clr = CyclicLR(base_lr=0.001, max_lr=0.006,step_size=4*(len(y_train)/batch_size))
callbacks = [clr, model_checkpointer]

#train model
hs=model_large.fit(x_train, y_train, batch_size=batch_size,verbose=1,
                    initial_epoch=92,epochs=92+16, callbacks=callbacks,
                    validation_data=(x_test, y_test))
model_large.save(model_file)
print('Saved model_small to disk')

In [ ]:
clr = CyclicLR(base_lr=0.004, max_lr=0.008,step_size=4*(len(y_train)/batch_size))
callbacks = [clr, model_checkpointer]

#train model
hs=model_large.fit(x_train, y_train, batch_size=batch_size,verbose=1,
                    initial_epoch=108,epochs=108+16, callbacks=callbacks,
                    validation_data=(x_test, y_test))
model_large.save(model_file)
print('Saved model_small to disk')

In [ ]:
clr = CyclicLR(base_lr=0.001, max_lr=0.008,step_size=4*(len(y_train)/batch_size))
callbacks = [clr, model_checkpointer]

#train model
hs=model_large.fit(x_train, y_train, batch_size=batch_size,verbose=1,
                    initial_epoch=124,epochs=124+24, callbacks=callbacks,
                    validation_data=(x_test, y_test))
model_large.save(model_file)
print('Saved model_small to disk')